# BAIT 509 Assignment 3

__Evaluates__: Class meetings 07, 08, and 09.

__Due__: Wednesday, March 28 at 10:00am (i.e., the start of Class Meeting 10).

## Instructions

You must use proper spelling and grammar.

## Exercise 1: Regression Beyond the Mean

In this exercise, we'll use the Bow River flow data. We've included it in the `data` folder of this assignment. Your task is to investigate the river flow from the day of the year (use `lubridate::yday` to get the day of year from a date).

For this exercise, you are free to use R.

### 1a Probabilistic Forecasting

Produce a probabilistic forecast of river flow on a day during "peak season" (i.e., when flow is large -- just eyeball it), and another on a day during the "low season". The estimates should take the form of a density function (plotted visually).

To produce a predictive distribution on any given day (say day `d`):

1. Subset the data that occur on day `d`.
2. Obtain the univariate sample of flow data from this subset.
3. Use ggplot2's `geom_density` on the subsetted univariate flow data. 

__Note__: this is the idea behind local methods in their simplest forms. 

Now, imagine you're interested in setting up a campground along the Bow River at this gauge location (the town of Banff is currently located there, but imagine it isn't). You've also learned that flows above 200 m^3/s typically result in a flood, with severity growing with larger flows. If you were to set up camp there, what does your predictive distribution tell you about the flooding situation you'd have to deal with year-in and year-out? Would you say this is a risky move?

### 1b Local Quantile Regression

Estimate the 0.1-quantile, mean, and 0.9-quantile on every day of the year, using the method mentioned in 4a. Specifically, to estimate a quantity on day `d`:

1. Subset the data that occur on day `d`.
2. Obtain the univariate sample of flow data from this subset.
3. Estimate the mean or quantile on this univariate sample. For quantile estimation, feel free to use the `quantile` function in R with any `type` argument. 

Hint: `dplyr` makes this easy -- just `group_by` the day of year, and calculate the estimates in the `summarize` function.


Plot your estimates of the 0.1-quantile, mean, and 0.9-quantile against the day of year, using `geom_line` to connect the dots for each of the three quantities. This will form your "regression curves": the middle curve being the mean regression curve, and the outer ones being quantile regression curves. Include the original data in this plot, too, but underneath your regression curves, and with quite a bit of alpha transparency. 

What you have is mean regression, with an 80% prediction interval given by the lower and upper curves.  

In 4a, you chose a day of the year corresponding to "peak season". What is your estimated 80% prediction interval of the river flow on this day? In addition, provide an interpretation of the 0.9-quantile. 

### 1c Crossing Quantile Curves

Using the local quantile estimation method in 4b, is it possible for quantile curves to "cross"? Why or why not?

### 1d Reduce Overfitting

The regression curves in 4b are overfit -- the curves are too wiggly, suggesting that there are patterns present in the data that aren't actually there. Re-make the plot of the regression curves in 4a, this time using `ggplot2`'s built-in local estimation capabilities. Choose smoothing parameters that look to give a fit that's "just right".

Hints:

- Don't worry about perfecting the fit here. If we really wanted to, we could choose it to optimize the model's score.
- Use `geom_smooth` for the mean regression curve, controlling the fit with the `span` parameter.
- Use `geom_quantile(method="rqss")` to fit a local quantile regression estimate, controlling the smoothing parameter with the `lambda` argument. Larger values of lambda correspond to wider windows; perhaps start your search with `lambda=10`.

## Exercise 2: Diving Deeper

Choose one of the methods introduced in Class Meeting 09 (SVM, Naive Bayes, ...). We only covered the basic idea behind each method, with the aim of giving you exposure to a variety of ideas. In this exercise, you'll dive a little deeper into a method of your choice. In particular, answer the following questions:

- Briefly research one way we can extend the method, that was not covered in class. What is the main idea behind the learning method? Briefly describe this (no more than a short paragraph). 
- What parameters/choices in the estimation control the bias-variance tradeoff, and how?
- When is this method generally useful, and in what situation might the extension you researched be useful?
- Apply this method, _with your researched extension_, to a dataset introduced in one of your assignments from this course.

## Exercise 6a: Naive Bayes and Laplace smoothing
rubric={reasoning:3}

The code below loads the newsgroups data from Lab 1 and runs Naive Bayes on it.

1. Explain why the conditional independence assumption is needed to make Naive Bayes a practical algorithm.
2. Discuss the `alpha` parameter of the sklearn classifier. What is it good for?
3. Try running the code with `alpha=0`. What happens? Why?
4. Try running the code with `alpha=100`. What happens? Why?

In [3]:
from sklearn.naive_bayes import BernoulliNB
import pickle 

with open('newsgroups.pkl', 'rb') as f:
    data = pickle.load(f)
Xtrain = data['X'].toarray() # use the dense version for simplicity 
ytrain = data['y']
Xtest = data['Xtest'].toarray()
ytest = data['ytest']
classnames = data['groupnames']
wordnames = data['wordlist']

In [4]:
model = BernoulliNB(alpha=1)
model.fit(Xtrain, ytrain)

pred_train = model.predict(Xtrain)
pred_test = model.predict(Xtest)
training_error = np.sum(pred_train != ytrain)/len(ytrain)
test_error = np.sum(pred_test != ytest)/len(ytest)
print('Training error = %f' % training_error)
print('Test error = %f' % test_error)

Training error = 0.201207
Test error = 0.187415


## (optional) Exercise 6b: Naive Bayes implementation
rubric={reasoning:1}

Below is an implementation of a naive Bayes classifier. While the `predict` function of the naive Bayes classifier is already implemented, the calculation of the variable $p\_xy$ is incorrect (right now, it just sets all values to $1/2$). 

1. Modify this function so that $p\_xy$ correctly computes the conditional probability of these values based on the frequencies in the data set. 
3. Compare your implmentation with [sklearn's implementation](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB).



In [32]:
import numpy as np


class NaiveBayes(): # only works when classes are 0,1,...,C-1

    def __init__(self, classnames, inputnames):
        self.classnames = classnames
        self.inputnames = inputnames
        self.C = len(classnames)
        
    # this implementation is incorrect (see comments below)
    def fit(self, X, y):
        N, D = X.shape
        C = self.C

        # Compute the probability of each class i.e p(y==c)
        counts = np.bincount(y)
        p_y = counts / N

        # Compute the conditional probabilities i.e.
        # p(x(i,j)=1 | y(i)==c) as p_xy
        # p(x(i,j)=0 | y(i)==c) as p_xy
        p_xy = np.zeros((D, C, 2)) # the 2 is because we have binary features, so x(i,j) can only take on 2 values
        for label in range(C): # loop through classes
            for feature in range(D): # loop through features
                # these two lines below are not right. replace them with the correct code.
                # it's not a lot of code; just a few lines.
                p_xy[feature, label, 0] = 0.5
                p_xy[feature, label, 1] = 0.5


        # Save parameters in model as dict
        
        self.p_y = p_y
        self.p_xy = p_xy

    def predict(self, X):
        N, D = X.shape
        C = self.C
        p_xy = self.p_xy
        p_y = self.p_y

        y_pred = np.zeros(N)

        for n in range(N):
            
            probs = p_y.copy()
            for d in range(D):
                probs *= p_xy[d, :, X[n, d]]

            y_pred[n] = np.argmax(probs)

        return y_pred
    
model = NaiveBayes(classnames=classnames, inputnames=wordnames)
print('Fitting...')
model.fit(X, y)
print('Predicting...')
pred_train = model.predict(X)
pred_test = model.predict(Xtest)
training_error = np.sum(pred_train != y)/len(y)
test_error = np.sum(pred_test != ytest)/len(ytest)
print('Training error = %f' % training_error)
print('Test error = %f' % test_error)

Fitting...
Predicting...
Training error = 0.666297
Test error = 0.661249


## Exercise 7a: quantitative comparison of classifiers
rubric={code:2,reasoning:3}

Here, we will compare the training and validation errors of the classifiers covered in this course:
  * $k$-nearest neighbours
  * decision trees
  * random forests
  * SVM
  * naive Bayes
  
For each classifier, use the scikit-learn implementation with default hyperparameters. 

Perform the comparison on the following 3 data sets:
  * handwritten digits (used in lab 1 and 2)
  * newsgroups (used in lab 1 and 2)
  * one other classification data set of your choice (for example, something you used in a previous course, something you found online, etc. Just give a brief explanation of where you got the data from.)

Report validation errors for all 15 classifier-dataset combinations. Discuss your results: 
 
  - Are certain classifiers better for certain problems? Why? 
  - Would you change any of the hyperparameters? 
  - What about speed... what would you do for a huge data set? 
  - Any other considerations?

Note: because all sklearn classifiers use the name fit/predict structure, you can put your 5 classifiers in a list/dict and iterate over them with a loop (and likewise with the data). This will probably be easier writing 15 calls to fit and 15 to predict, etc. For example:

In [22]:
classifiers = {
    'knn'           : KNeighborsClassifier(),
    'decision tree' : DecisionTreeClassifier(),
    'random forest' : RandomForestClassifier(),
    'SVM'           : SVC(),
    'naive Bayes'   : BernoulliNB() 
}

Note: you'll have to be a bit careful with `BernoulliNB`, as it's expecting binary features. You can skip it when it's not applicable, or you can try binarizing the features. Your call.

### Exercise 7b: qualitative comparison of classifiers 
rubric={reasoning:4}

Fill in the following table with at least one entry per box.

Classifier |      Strengths | Weaknesses | Key hyperparameters |
-----------|      ------------|------------|---------------------|
$k$-NN              |            |            |                     |
decision tree       |            |            |                     |
random forest |            |            |                     |
SVM                 |            |            |                     |
naive Bayes         |            |            |                     |

For strengths and weaknesses, some things to consider are:
  * ease of use for multi-class classification
  * concerns about underfitting
  * concerns about overfitting
  * speed
  * scalability for large data sets
  * effectiveness when number of examples $\ll$ number of features
  * effectiveness when number of examples $\gg$ number of features
  * ability to represent uncertainty
  * etc.

## Exercise 8: data scaling
rubric={reasoning:3}

Of the five classification methods listed below, which ones are sensitive to the normalization of the features? For example, if one of your features is length, which methods would give different results if the unit of length was changed from metres to centimetres across all the (training and testing) data? For each method, answer "yes" or "no" or "not applicable" and provide a one-sentence explanation. 

Note: you are welcome to perform empirical tests, although it should be possible to answer the question using your knowledge of these classifiers. If testing things out empirically, be careful that your software package (e.g. sklearn) isn't normalizing things by default.

Classifier | Sensitive? | Short explanation |
-----------| ----------|---------------------|
kNN                  |            |
decision tree |            |            | 
random forest |            |            |
SVM          |            |            |  
naive Bayes  |            |            | 


-----------------
__Attribution__: Much of this material is from Mark Schmidt via CPSC 340.